In [22]:
from pyspark.sql import SparkSession, Row
spark=SparkSession.builder.getOrCreate()
from pyspark.sql.functions import col, substring, expr
from pyspark.sql.types import DoubleType, LongType

In [23]:
#lendo os arquivos json com os dados de VRA

df=spark.read.json(['C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20211.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20212.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20213.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20214.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20215.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20216.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20217.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20218.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_20219.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_202110.json', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_202111.json'])

In [ ]:
#estrutura do df VRA

df.printSchema()

In [ ]:
#visualizando as primeiras 200 linhas do df VRA

df.show(200)

In [26]:
#criando um novo df com o nome das colunas normalizados para snake case

df_vra=df.withColumnRenamed("ChegadaPrevista", "Chegada_Prevista").withColumnRenamed("ChegadaReal", "Chegada_Real").withColumnRenamed("CódigoAutorização","Codigo_Autorizacao")\
    .withColumnRenamed("CódigoJustificativa", "Codigo_Justificativa").withColumnRenamed("CódigoTipoLinha", "Codigo_Tipo_Linha").withColumnRenamed("ICAOAeródromoDestino", "ICAO_Aerodromo_Destino")\
    .withColumnRenamed("ICAOAeródromoOrigem", "ICAO_Aerodromo_Origem").withColumnRenamed("ICAOEmpresaAérea", "ICAO_Empresa_Aerea").withColumnRenamed("NúmeroVoo", "Numero_Voo")\
    .withColumnRenamed("PartidaPrevista", "Partida_Prevista").withColumnRenamed("PartidaReal", "Partida_Real").withColumnRenamed("SituaçãoVoo", "Situacao_Voo")

In [ ]:
#estrutura do novo df VRA com o nome das colunas normalizados

df_vra.printSchema()

In [28]:
#escrevendo e salvando o novo df VRA em parquet e com modo overwrite

df_vra.write.mode("overwrite").parquet("C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/VRA/VRA_2021_GERAL.parquet")

In [29]:
#lendo os arquivos csv com os dados de AIR_CIA

df1=spark.read.options(header='True', delimiter=';', inferSchema='True').csv(['C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/AIR_CIA/ANAC_20211220_203627.csv', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/AIR_CIA/ANAC_20211220_203643.csv', 'C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/AIR_CIA/ANAC_20211220_203733.csv'])

In [ ]:
#estrutura do df AIR_CIA

df1.printSchema()

In [ ]:
#visualizando as linhas do df AIR_CIA

df1.show(50)

In [113]:
#criando um novo df para os dados da AIRCIA com o nome das colunas padronizados em snake case

df_AIRCIA=df1.withColumnRenamed("Razão Social", "Razao_Social").withColumnRenamed("ICAO IATA", "ICAO_IATA").withColumnRenamed("Atividades Aéreas", "Atividades_Aereas").withColumnRenamed("Endereço Sede", "Endereco_Sede")\
    .withColumnRenamed("E-Mail", "E_Mail").withColumnRenamed("Decisão Operacional", "Decisao_Operacional").withColumnRenamed("Data Decisão Operacional", "Data_Decisao_Operacional").withColumnRenamed("Validade Operacional", "Validade_Operacional")

In [ ]:
#visualizando as linhas do novo df

df_AIRCIA.show(200)

In [114]:
#criando um novo df para os dados AIR_CIA separando a coluna ICAO_IATA em duas novas colunas com um case when para tratamento de acordo com a maneira que o dado da linha vier

df_AIRCIA_2=df_AIRCIA.withColumn("ICAO", substring("ICAO_IATA",1, 3)).withColumn("IATA", expr("CASE WHEN ICAO_IATA like '%%% %%' THEN substring(ICAO_IATA, 5, 2)" + "ELSE NULL END"))

In [ ]:
#visualizando os dados do df com as colunas em snake case e com as duas novas colunas

df_AIRCIA_2.show(50)

In [37]:
#escrevendo e salvando df AIRCIA unificado e tratado em parquet e com modo overwrite

df_AIRCIA_2.write.mode("overwrite").parquet("C:/Users/Murillo/Desktop/DADOS TESTE_PROJETAS/AIR_CIA/AIR_CIA_GERAL")

In [134]:
##transformando o df AIRCIA em uma tabela temporaria para dar inicio a manipulacao de dados para gerar a primeira view

df_AIRCIA_2.createOrReplaceTempView("aircia_origem")

#removendo dados duplicados aircia
df_aircia=spark.sql("SELECT DISTINCT * FROM aircia_origem")
df_aircia.createOrReplaceTempView("aircia")

#transformando o df VRA em uma tabela temporaria para dar inicio a manipulacao de dados para gerar a primeira view
df_vra.createOrReplaceTempView("vra")


########################################################
#df_api.createOrReplaceTempView("api_aeroportos")
########################################################

In [135]:
#criacao de tabelas temporarias e manipulacao de dados com SQL para gerar ao final de tudo (df_vra_temp4) com um df com dados das rotas(ICAO aeroporto origem + ICAO aeroporto destino) mais utilizadas por cada empresa aerea

df_vra_temp=spark.sql("""
SELECT 
    ICAO_Empresa_Aerea, concat(ICAO_Aerodromo_Origem, ICAO_Aerodromo_Destino) as Rota 
FROM 
    vra 
WHERE 
    Situacao_Voo = 'REALIZADO' 
ORDER BY 
    ICAO_Empresa_Aerea, Rota""")
    
df_vra_temp.createOrReplaceTempView("vra_temp")


df_vra_temp2=spark.sql("""
SELECT 
    ICAO_Empresa_Aerea, Rota, count(Rota) as Quantidade 
FROM 
    vra_temp 
GROUP BY 
    ICAO_Empresa_Aerea, Rota 
ORDER BY 
    ICAO_Empresa_Aerea""")

df_vra_temp2.createOrReplaceTempView("vra_temp2")


df_vra_temp3=spark.sql("""
SELECT 
    ICAO_Empresa_Aerea, MAX(Quantidade) as Qtd_MAX 
FROM 
    vra_temp2 
GROUP BY 
    ICAO_Empresa_Aerea 
ORDER BY 
    ICAO_Empresa_Aerea""")

df_vra_temp3.createOrReplaceTempView("vra_temp3")


df_vra_temp4=spark.sql("""
SELECT 
    vra_temp2.ICAO_Empresa_Aerea, vra_temp2.Rota, vra_temp2.Quantidade 
FROM 
    vra_temp2 
INNER JOIN 
    vra_temp3 ON vra_temp2.ICAO_Empresa_Aerea = vra_temp3.ICAO_Empresa_Aerea 
WHERE 
    vra_temp2.Quantidade = vra_temp3.Qtd_MAX""")

df_vra_temp4.createOrReplaceTempView("vra_temp4")






In [ ]:
df_vra_temp4.show()

In [ ]:
#gerando uma tabela final com dados de AIR CIA contendo a rota(Aeroporto Origem -> Aeroporto Destino) feita com maior frequencia por cada empresa aerea

df_aircia_final=spark.sql("""
SELECT 
    Razao_Social, ICAO, IATA, CNPJ, Atividades_Aereas, Endereco_Sede, Telefone, E_Mail, Decisao_Operacional, Data_Decisao_Operacional, Validade_Operacional, substring(vra_temp4.Rota, 1, 4) as Aeroporto_Origem, substring(vra_temp4.Rota, 5, 4) as Aeroporto_Destino, vra_temp4.Quantidade 
FROM 
    aircia 
LEFT JOIN 
    vra_temp4 ON aircia.ICAO = vra_temp4.ICAO_Empresa_Aerea""")

df_aircia_final.createOrReplaceTempView("aircia_final")

df_aircia_final.show()


In [ ]:
#criando um dataframe com os dados da view1

#algumas empresas aereas chegaram ao final com seu destino mais frequente com dados nulos por 2 motivos:
# 1: A empresa PRIME TÁXI AIR LTDA não possui o dado de ICAO na tabela de AIR CIA de origem
# 2: As outras empresas não possuem dados na tabela de VRA com os respectivos ICAO representados pela tabela AIR CIA

df_view1=spark.sql("SELECT aircia_final.Razao_Social, aircia_final.Aeroporto_Origem, aircia_final.Aeroporto_Destino FROM aircia_final")


In [171]:
#df_vra.show()
#df_aircia.show()

df_view2_temp1=spark.sql("""
SELECT 
    vra.ICAO_Aerodromo_Origem, vra.ICAO_Aerodromo_Destino, vra.ICAO_Empresa_Aerea, aircia.Razao_Social, vra.Numero_Voo, vra.Situacao_Voo 
FROM 
    vra 
INNER JOIN 
    aircia ON vra.ICAO_Empresa_Aerea = aircia.ICAO
WHERE
    vra.Situacao_voo = 'REALIZADO'""")

df_view2_temp1.createOrReplaceTempView("view2_temp1")

df_view2_temp1.show()

+---------------------+----------------------+------------------+--------------------+----------+------------+
|ICAO_Aerodromo_Origem|ICAO_Aerodromo_Destino|ICAO_Empresa_Aerea|        Razao_Social|Numero_Voo|Situacao_Voo|
+---------------------+----------------------+------------------+--------------------+----------+------------+
|                 SBGR|                  SBAR|               TAM|TAM LINHAS AÉREAS...|      3268|   REALIZADO|
|                 SBGR|                  SBAR|               TAM|TAM LINHAS AÉREAS...|      3268|   REALIZADO|
|                 SBGR|                  SBAR|               TAM|TAM LINHAS AÉREAS...|      3268|   REALIZADO|
|                 SBGR|                  SBAR|               TAM|TAM LINHAS AÉREAS...|      3268|   REALIZADO|
|                 SBGR|                  SBAR|               TAM|TAM LINHAS AÉREAS...|      3268|   REALIZADO|
|                 SBGR|                  SBAR|               TAM|TAM LINHAS AÉREAS...|      3268|   REALIZADO|
|

In [188]:

############################          MANIPULACAO PARA DADOS DE AEROPORTO DE ORIGEM              ###################################
df_view2_temp2=spark.sql("SELECT ICAO_Aerodromo_Origem, Razao_Social, COUNT(Razao_Social) as Qtd_Origem FROM view2_temp1 GROUP BY ICAO_Aerodromo_Origem, Razao_Social ORDER BY ICAO_Aerodromo_Origem")
df_view2_temp2.createOrReplaceTempView("view2_temp2")



df_view2_temp2_maxOrigem=spark.sql("SELECT ICAO_Aerodromo_Origem, MAX(Qtd_Origem) as Qtd_Origem FROM view2_temp2 GROUP BY ICAO_Aerodromo_Origem")
df_view2_temp2_maxOrigem.createOrReplaceTempView("view2_temp2_maxOrigem")



df_view2_temp2_finalOrigem=spark.sql("""
SELECT 
    view2_temp2.ICAO_Aerodromo_Origem, view2_temp2.Qtd_Origem, view2_temp2.Razao_Social 
FROM 
    view2_temp2 
INNER JOIN 
    view2_temp2_maxOrigem ON view2_temp2.ICAO_Aerodromo_Origem = view2_temp2_maxOrigem.ICAO_Aerodromo_Origem 
WHERE 
    view2_temp2.Qtd_Origem = view2_temp2_maxOrigem.Qtd_Origem""")
df_view2_temp2_finalOrigem.createOrReplaceTempView("view2_temp2_finalOrigem")


############################          MANIPULACAO PARA DADOS DE AEROPORTO DE DESTINO              ###################################
df_view2_temp3=spark.sql("SELECT ICAO_Aerodromo_Destino, Razao_Social, COUNT(Razao_Social) as Qtd_Destino FROM view2_temp1 GROUP BY ICAO_Aerodromo_Destino, Razao_Social ORDER BY ICAO_Aerodromo_Destino")
df_view2_temp3.createOrReplaceTempView("view2_temp3")



df_view2_temp3_maxDestino=spark.sql("SELECT ICAO_Aerodromo_Destino, MAX(Qtd_Destino) as Qtd_Destino FROM view2_temp3 GROUP BY ICAO_Aerodromo_Destino")
df_view2_temp3_maxDestino.createOrReplaceTempView("view2_temp3_maxDestino")



df_view2_temp3_finalDestino=spark.sql("""
SELECT 
    view2_temp3.ICAO_Aerodromo_Destino, view2_temp3.Qtd_Destino, view2_temp3.Razao_Social 
FROM 
    view2_temp3 
INNER JOIN 
    view2_temp3_maxDestino ON view2_temp3.ICAO_Aerodromo_Destino = view2_temp3_maxDestino.ICAO_Aerodromo_Destino 
WHERE 
    view2_temp3.Qtd_Destino = view2_temp3_maxDestino.Qtd_Destino""")
df_view2_temp3_finalDestino.createOrReplaceTempView("view2_temp3_finalDestino")





+----------------------+-----------+--------------------+
|ICAO_Aerodromo_Destino|Qtd_Destino|        Razao_Social|
+----------------------+-----------+--------------------+
|                  KORD|         13|TAM LINHAS AÉREAS...|
+----------------------+-----------+--------------------+



In [204]:
############################          UNIFICANDO DADOS DE QUANTIDADE DE ORIGEM E DESTINO             ###################################
#RIGHT JOIN devido a um caso especifico de um aeroporto que só recebeu pouso

df_view2_qtdUnificada=spark.sql("""
SELECT 
    view2_temp3_finalDestino.ICAO_Aerodromo_Destino as ICAO_Aerodromo, view2_temp3_finalDestino.Razao_Social, view2_temp2_finalOrigem.Qtd_Origem AS Qtd_Decolagem, view2_temp3_finalDestino.Qtd_Destino AS Qtd_Pouso, (view2_temp3_finalDestino.Qtd_Destino+view2_temp2_finalOrigem.Qtd_Origem) AS Qtd_Total 
FROM 
    view2_temp2_finalOrigem 
RIGHT JOIN 
    view2_temp3_finalDestino ON view2_temp2_finalOrigem.ICAO_Aerodromo_Origem = view2_temp3_finalDestino.ICAO_Aerodromo_Destino""")

df_view2_qtdUnificada.show(1000)


+--------------+--------------------+-------------+---------+---------+
|ICAO_Aerodromo|        Razao_Social|Qtd_Decolagem|Qtd_Pouso|Qtd_Total|
+--------------+--------------------+-------------+---------+---------+
|          SBPS|AZUL LINHAS AÉREA...|         1927|     1929|     3856|
|          SBIH|MAP TRANSPORTES A...|          226|      225|      451|
|          SBSJ|SIDERAL LINHAS AÉ...|            1|        1|        2|
|          SBSJ|OMNI TÁXI AÉREO S.A.|            1|        1|        2|
|          SSSC|AZUL CONECTA LTDA...|            7|        7|       14|
|          SBCH|AZUL LINHAS AÉREA...|          809|      808|     1617|
|          SCCF|ABSA - AEROLINHAS...|            1|        1|        2|
|          SBCR|AZUL LINHAS AÉREA...|          166|      161|      327|
|          SNEE|AZUL CONECTA LTDA...|            8|        9|       17|
|          OMAA|ABSA - AEROLINHAS...|            1|        2|        3|
|          SBUG|AZUL LINHAS AÉREA...|          165|      164|   